# Models Work

In [2]:
!git clone https://github.com/JulianKrese/project_chd

Cloning into 'project_chd'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 38 (delta 5), reused 2 (delta 2), pack-reused 29
Receiving objects: 100% (38/38), 1.52 MiB | 5.49 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


X_train = pd.read_csv("/content/project_chd/X_train_df.csv")
y_train = pd.read_csv("/content/project_chd/y_train_df.csv")

# getting test sets
X_test = pd.read_csv("/content/project_chd/X_test_df.csv")
y_test = pd.read_csv("/content/project_chd/y_test_df.csv")


# trying linear regression with all variables
stock_reg = LinearRegression().fit(X_train, y_train)
print("R^2 = ", stock_reg.score(X_test, y_test))

# trying linear regression with min-max normalizing


# trying linear regression with choosing correlated variables


# trying linear regression with polynomial expanders (probably want to test multiple degrees)




   Unnamed: 0  sex  age  currentSmoker  cigsPerDay  BPMeds  prevalentStroke  \
0        1267    1   58              0           0       0                0   
1        1209    0   40              1          15       0                0   
2        1183    1   38              1          43       0                0   
3        3225    0   43              0           0       0                0   
4        1054    0   54              0           0       0                0   

   prevalentHyp  diabetes  totChol  sysBP  diaBP    BMI  heartRate  glucose  \
0             0         0      220  143.0  104.0  29.85         75       87   
1             0         0      199  122.0   82.0  22.16         85       77   
2             1         0      170  130.0   94.0  23.90        110       75   
3             0         0      202  124.0   92.0  21.26         75       74   
4             1         0      237  171.5  105.5  34.25         91      104   

   edu_some_hs  edu_hs/ged  edu_some_college/voc  